In [1]:
!pip install tf2onnx


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [12]:
import tensorflow as tf
import tf2onnx
import onnx

import boto3
from botocore.client import Config
from botocore.exceptions import ClientError
import datetime
import os

In [5]:
h5_model_path = "fracture_model.h5"
model = tf.keras.models.load_model(h5_model_path)

input_signature = [tf.TensorSpec([1, 256, 256, 3], tf.float32, name='x')]
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=13)

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = f"classification_model_{timestamp}.onnx"

onnx.save(onnx_model, model_name)

2024-08-19 09:41:58.425163: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-08-19 09:41:58.425298: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session
2024-08-19 09:42:00.314587: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2024-08-19 09:42:00.314729: I tensorflow/core/grappler/clusters/single_machine.cc:361] Starting new session


In [6]:
# MinIO connection details
minio_endpoint = os.getenv('AWS_S3_ENDPOINT')
access_key = os.getenv('AWS_ACCESS_KEY_ID')
secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
bucket_name = "models"
region = os.getenv('AWS_DEFAULT_REGION')

# Initialize the S3 client
s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version='s3v4')
)

In [8]:
try:
    response = s3_client.upload_file(model_name, 'models', f"fracture/{model_name}")
except Error as e:
    logging.error(e)